## Week 3 Assignment

#### by Megan

### To get dataset from Wikipedia

In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans 

from geopy.geocoders import Nominatim
import requests

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs=pd.read_html(url)
print(len(dfs))

3


In [3]:
df=dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Clean Dataset

In [4]:
df['Neighbourhood']=df['Neighbourhood'].replace('Not assigned', df['Borough'])
df=df[df['Borough']!='Not assigned']
df.reset_index(drop=True).head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
df.shape

(103, 3)

In [6]:
df_postcode = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_postcode.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Read csv file to get Latitude and Longitude and merge to Postal Code dataset

In [7]:
df_Lat_Lon=pd.read_csv("Geospatial_Coordinates.csv")
df_Lat_Lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_merge=pd.merge(left=df,right=df_Lat_Lon,left_on="Postal Code",right_on="Postal Code")
df_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Call Foursquare API to get nearby venues

In [9]:
CLIENT_ID = 'Y5YLNIDNPUI23JM2Q5BYXBKVWNHGV3JIL51XMEENCTF0RQDB' # your Foursquare ID
CLIENT_SECRET = '4AU0BET30XXBJ2HOLOJE0XUBD4PTBYAKIY1FHPQXXJW1LCYP' # your Foursquare Secret
#ACCESS_TOKEN = 'YYU1ENYS03KWZXR5CJNZPPM0LS005NUJJAGVKPYYUDJHWRMR' # your FourSquare Access Token
VERSION = '20210104'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
latitude=43.753259
longitude=-79.329656
search_query="Toronto"
radius=500

Your credentails:
CLIENT_ID: Y5YLNIDNPUI23JM2Q5BYXBKVWNHGV3JIL51XMEENCTF0RQDB
CLIENT_SECRET:4AU0BET30XXBJ2HOLOJE0XUBD4PTBYAKIY1FHPQXXJW1LCYP


In [10]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Y5YLNIDNPUI23JM2Q5BYXBKVWNHGV3JIL51XMEENCTF0RQDB&client_secret=4AU0BET30XXBJ2HOLOJE0XUBD4PTBYAKIY1FHPQXXJW1LCYP&ll=43.753259,-79.329656&v=20210104&query=Toronto&radius=500&limit=30'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600be0083858e2550cd2712c'},
 'response': {'venues': [{'id': '59e5fd0950a6f02796cad187',
    'name': 'Emergency Plumbing Toronto Inc.',
    'location': {'address': '44 Valleywoods Road',
     'lat': 43.754453590651806,
     'lng': -79.3333187699318,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.754453590651806,
       'lng': -79.3333187699318}],
     'distance': 323,
     'postalCode': 'M3A 2R6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['44 Valleywoods Road',
      'Toronto ON M3A 2R6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d124941735',
      'name': 'Office',
      'pluralName': 'Offices',
      'shortName': 'Office',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1611390984',
    'hasPerk': False},
   {'id': '54f49a87498e59d2129e83f7

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
Toronto_venues=getNearbyVenues(df_merge['Neighbourhood'], latitudes=df_merge['Latitude'], longitudes=df_merge['Longitude'], radius=500)

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [14]:
Toronto_venues.shape

(1329, 7)

In [15]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Towns On The Ravine,43.754754,-79.332552,Hotel
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [16]:
Toronto_Category=pd.get_dummies(Toronto_venues[["Venue Category"]],prefix="",prefix_sep="")
N=Toronto_venues['Neighborhood']
Toronto_Category.drop(['Neighborhood'],axis=1,inplace=True)
Toronto_Category.insert(0,'Neighborhood',N)
Toronto_Category.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
Toronto_group=Toronto_Category.groupby('Neighborhood').mean().reset_index()
Toronto_group.shape

(95, 236)

### Cluster Neighborhood

In [18]:
k=5
kmean=KMeans(init="k-means++",n_clusters=k,n_init=12)
Toronto_Cluster=Toronto_group.drop('Neighborhood',axis=1)
kmean.fit(Toronto_Cluster)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=12, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [19]:
kmean.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 2,
       0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 4, 0,
       2, 0, 0, 0, 0, 0, 2])

In [20]:
Toronto_group.insert(0,"Cluster Ind",kmean.labels_)
Toronto_group.head()

,Cluster Ind,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
Toronto_final=pd.merge(left=df_merge,right=Toronto_group,left_on="Neighbourhood",right_on="Neighborhood")
Toronto_final.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Ind,Neighborhood,Accessories Store,Adult Boutique,Airport,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Parkwoods,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Victoria Village,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,"Regent Park, Harbourfront",0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,"Lawrence Manor, Lawrence Heights",0.142857,0.0,0.0,...,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.000000
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,"Queen's Park, Ontario Provincial Government",0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333


### Visualization

In [22]:
address = 'Toronto,Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
map_cluster=folium.Map(location=[43.6534817, -79.3839347],width=750, height=500, zoom_start=10)
# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to map
for lat, lng, neighborhood,cluster in zip(Toronto_final['Latitude'], Toronto_final['Longitude'], Toronto_final['Neighbourhood'],Toronto_final["Cluster Ind"]):
    #label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(str(neighborhood)+' Cluster '+str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster)  
    
map_cluster



### Examine Cluster

In [24]:
#Cluster 1
Toronto_final.loc[Toronto_final['Cluster Ind'] == 0, Toronto_final.columns[[1] + list(range(5, Toronto_final.shape[1]))]]

,Borough,Cluster Ind,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
1,North York,0,Victoria Village,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000
2,Downtown Toronto,0,"Regent Park, Harbourfront",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.033333
3,North York,0,"Lawrence Manor, Lawrence Heights",0.142857,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.0000,0.0,0.000000
4,Downtown Toronto,0,"Queen's Park, Ontario Provincial Government",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.033333
5,Scarborough,0,"Malvern, Rouge",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000
6,North York,0,Don Mills,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000
7,North York,0,Don Mills,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000
8,East York,0,"Parkview Hill, Woodbine Gardens",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000
9,Downtown Toronto,0,"Garden District, Ryerson",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000
10,North York,0,Glencairn,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000


In [25]:
#Cluster 2
Toronto_final.loc[Toronto_final['Cluster Ind'] == 1, Toronto_final.columns[[1] + list(range(5, Toronto_final.shape[1]))]]

,Borough,Cluster Ind,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
55,North York,1,"Humberlea, Emery",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Etobicoke,1,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
#Cluster 3
Toronto_final.loc[Toronto_final['Cluster Ind'] == 2, Toronto_final.columns[[1] + list(range(5, Toronto_final.shape[1]))]]

,Borough,Cluster Ind,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,North York,2,Parkwoods,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
12,Scarborough,2,"Rouge Hill, Port Union, Highland Creek",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
20,York,2,Caledonia-Fairbanks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0
34,East York,2,"East Toronto, Broadview North (Old East York)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
58,Central Toronto,2,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
61,York,2,Weston,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
63,North York,2,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
82,Scarborough,2,"Milliken, Agincourt North, Steeles East, L'Amo...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
88,Downtown Toronto,2,Rosedale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
94,Etobicoke,2,"The Kingsway, Montgomery Road, Old Mill North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [27]:
#Cluster 4
Toronto_final.loc[Toronto_final['Cluster Ind'] == 3, Toronto_final.columns[[1] + list(range(5, Toronto_final.shape[1]))]]

,Borough,Cluster Ind,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
31,Scarborough,3,Scarborough Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,Central Toronto,3,"Moore Park, Summerhill East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#Cluster 5
Toronto_final.loc[Toronto_final['Cluster Ind'] == 4, Toronto_final.columns[[1] + list(range(5, Toronto_final.shape[1]))]]

,Borough,Cluster Ind,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
11,Etobicoke,4,"West Deane Park, Princess Gardens, Martin Grov...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
